Saai Narayann Maddu

210968036

Section A1

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('CliffWalking-v0')


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
def monte_carlo_exploring_starts(env, num_episodes=500):
    """
    Performs Monte Carlo with Exploring Starts algorithm to estimate the optimal policy.

    Args:
    - env: The environment object representing the Markov Decision Process (MDP).
    - num_episodes: Number of episodes to run.

    Returns:
    - optimal_policy: The optimal policy derived from Q values.
    - Q_values: The state-action value function.
    - total_steps: List containing the number of steps taken in each episode.
    """

    # Initialize Q values and N counts for each state-action pair
    Q_values = np.zeros((env.observation_space.n, env.action_space.n))
    N_counts = np.zeros((env.observation_space.n, env.action_space.n))

    gamma = 1.0  # discount factor
    total_steps = []  # List to store the number of steps taken in each episode

    # Iterate over episodes
    for i in range(num_episodes):
        state = env.reset()  # Initialize the environment and get the initial state
        episode = []  # List to store the sequence of (state, action, reward) tuples for the episode
        done = False
        steps = 0  # Counter to track the number of steps taken in the episode

        # Generate an episode using exploring starts
        while not done:
            action = np.random.choice(env.action_space.n)  # Choose a random action
            next_state, reward, done, info = env.step(action)  # Take the chosen action
            episode.append((state, action, reward))  # Record the transition
            state = next_state  # Update the current state
            steps += 1  # Increment the step counter

        total_steps.append(steps)  # Record the total number of steps taken in the episode

        # Update Q values using the episode
        returns = 0
        for j in range(len(episode) - 1, -1, -1):  # Iterate over the episode in reverse order
            state, action, reward = episode[j]
            returns = gamma * returns + reward  # Compute the return
            N_counts[state][action] += 1  # Increment the visit count for the state-action pair
            Q_values[state][action] += (returns - Q_values[state][action]) / N_counts[state][action]  # Update Q value

    # Derive optimal policy from Q values
    optimal_policy = np.argmax(Q_values, axis=1)

    return optimal_policy, Q_values, total_steps


In [5]:
def on_policy_monte_carlo_control(env, num_episodes=500, epsilon=0.1):
    """
    Performs On-Policy Monte Carlo Control with Ɛ-greedy policy to estimate the optimal policy.

    Args:
    - env: The environment object representing the Markov Decision Process (MDP).
    - num_episodes: Number of episodes to run.
    - epsilon: The probability of choosing a random action (exploration).

    Returns:
    - optimal_policy: The optimal policy derived from Q values.
    - Q_values: The state-action value function.
    - total_steps: List containing the number of steps taken in each episode.
    """

    # Initialize Q values and N counts for each state-action pair
    Q_values = np.zeros((env.observation_space.n, env.action_space.n))
    N_counts = np.zeros((env.observation_space.n, env.action_space.n))

    gamma = 1.0  # discount factor
    total_steps = []  # List to store the number of steps taken in each episode

    # Iterate over episodes
    for i in range(num_episodes):
        state = env.reset()  # Initialize the environment and get the initial state
        done = False
        steps = 0  # Counter to track the number of steps taken in the episode

        # Generate an episode using Ɛ-greedy policy
        while not done:
            if np.random.uniform(0, 1) < epsilon:
                action = env.action_space.sample()  # Choose a random action
            else:
                action = np.argmax(Q_values[state])  # Choose the action with maximum Q value
            next_state, reward, done, info = env.step(action)  # Take the chosen action
            N_counts[state][action] += 1  # Increment the visit count for the state-action pair
            Q_values[state][action] += (reward + gamma * np.max(Q_values[next_state]) - Q_values[state][action]) / N_counts[state][action]  # Update Q value
            state = next_state  # Update the current state
            steps += 1  # Increment the step counter

        total_steps.append(steps)  # Record the total number of steps taken in the episode

    # Derive optimal policy from Q values
    optimal_policy = np.argmax(Q_values, axis=1)

    return optimal_policy, Q_values, total_steps


In [6]:
monte_carlo_es_policy, monte_carlo_es_Q_values, total_steps_es = monte_carlo_exploring_starts(env)
on_policy_mc_control_policy, on_policy_mc_control_Q_values, total_steps_control = on_policy_monte_carlo_control(env)


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [7]:
print(str.format('Total Number of Steps taken to reach Optimal Policy using Monte Carlo ES: {}', sum(total_steps_es)))
print(str.format('Total Number of Steps taken to reach Optimal Policy using On-Policy First-Visit MC Control: {}', sum(total_steps_control)))

Total Number of Steps taken to reach Optimal Policy using Monte Carlo ES: 3157424
Total Number of Steps taken to reach Optimal Policy using On-Policy First-Visit MC Control: 17375


In [8]:
print(str.format('Average Number of Steps per Episode taken to reach Optimal Policy using Monte Carlo ES: {}', sum(total_steps_es)/len(total_steps_es)))
print(str.format('Average Number of Steps per Episode taken to reach Optimal Policy using On-Policy First-Visit MC Control: {}', sum(total_steps_control)/len(total_steps_control)))



Average Number of Steps per Episode taken to reach Optimal Policy using Monte Carlo ES: 6314.848
Average Number of Steps per Episode taken to reach Optimal Policy using On-Policy First-Visit MC Control: 34.75
